In [ ]:
# Dependency Installation Strategy for Google Colab (Python 3.12, Nov 2025)
# - Google Colab now uses Python 3.12 with pre-installed TensorFlow 2.19+ ecosystem
# - Must work with Colab's native package versions to avoid binary incompatibilities
# - Strategy: Clean conflicting packages, then install only what's needed
# - Let pip resolve compatible versions automatically (no forced pinning)

# Step 1: Remove conflicting pre-installed packages
print('Removing conflicting packages...')
!pip uninstall -y tensorflow-decision-forests tensorflowjs tensorflow-text tf-keras ydf -qq

# Step 2: Reinstall TensorFlow ecosystem with compatible versions
print('\nReinstalling TensorFlow stack (this may take 2-3 minutes)...')
!pip install -q --upgrade --force-reinstall \
    "tensorflow>=2.17.0" \
    "tensorflowjs>=4.20.0" \
    "mediapipe>=0.10.14" \
    "protobuf>=4.25.0,<5.0.0"

# Step 3: Install additional ML libraries
print('\nInstalling additional libraries...')
!pip install -q \
    "kagglehub" \
    "scikit-learn" \
    "matplotlib" \
    "seaborn" \
    "tqdm" \
    "opencv-python-headless"

print('\n✓ Installation complete!\n')

# Verify critical package versions
import sys
print(f"Python version: {sys.version}")

import tensorflow as tf
import mediapipe as mp
import google.protobuf
import numpy as np
import pandas as pd

print(f"\n✓ TensorFlow version: {tf.__version__}")
print(f"✓ MediaPipe version: {mp.__version__}")
print(f"✓ Protobuf version: {google.protobuf.__version__}")
print(f"✓ NumPy version: {np.__version__}")
print(f"✓ Pandas version: {pd.__version__}")

# Check for GPU availability
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"\n✓ GPU Available: {len(gpus)} GPU(s) detected")
    for gpu in gpus:
        print(f"  - {gpu.name}")
else:
    print("\n⚠ No GPU detected - training will use CPU (slower)")


In [ ]:
# Step 2: Mount Drive and configure paths/seeds
import os
import random
import json
import shutil
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf

# - Mount Google Drive with error handling
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    print('✓ Google Drive mounted successfully')
except Exception as e:
    print(f'⚠ Warning: Could not mount Drive: {e}')
    print('  Continuing without Drive - artifacts will not persist after session ends')

# - Fix seeds for fully reproducible splits and training
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.keras.utils.set_random_seed(SEED)
# Enable deterministic operations for full reproducibility
tf.config.experimental.enable_op_determinism()
print(f'✓ Random seeds set to {SEED} for reproducibility')

# - Declare canonical paths used throughout the pipeline
DATASET_KAGGLE = 'kapillondhe/american-sign-language'
COLAB_EXTRACT_DIR = '/kaggle/input/american-sign-language/ASL_Dataset'
OUTPUT_DIR = '/content/drive/MyDrive/sign_language_training'
RAW_LANDMARKS_NPZ = os.path.join(OUTPUT_DIR, 'hand_landmarks_raw.npz')
PROC_NPZ = os.path.join(OUTPUT_DIR, 'hand_landmarks_processed.npz')
BEST_H5 = os.path.join(OUTPUT_DIR, 'best_model.h5')
BEST_KERAS = os.path.join(OUTPUT_DIR, 'best_model.keras')
SAVED_MODEL_DIR = os.path.join(OUTPUT_DIR, 'saved_model')
TFJS_DIR = os.path.join(OUTPUT_DIR, 'tfjs_model')
LABELS_JSON = os.path.join(OUTPUT_DIR, 'labels.json')

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f'\n✓ Output directory: {OUTPUT_DIR}')
print(f'✓ Dataset will be downloaded to: {COLAB_EXTRACT_DIR}')


In [ ]:
# Step 3: Download ASL dataset via kagglehub and validate structure
import kagglehub
from pathlib import Path

print('Downloading dataset from Kaggle...')
print(f'Dataset: {DATASET_KAGGLE}')

try:
    # - Pull the dataset via kagglehub (handles authentication automatically in Colab)
    path = kagglehub.dataset_download(DATASET_KAGGLE)
    print(f'✓ kagglehub download complete: {path}')
except Exception as e:
    print(f'❌ Error downloading dataset: {e}')
    print('⚠ Make sure you have internet connection in Colab')
    raise

# Use the actual kagglehub download path, not the hardcoded COLAB_EXTRACT_DIR
dataset_root = Path(path)

# Check if ASL_Dataset subdirectory exists
if (dataset_root / 'ASL_Dataset').exists():
    dataset_root = dataset_root / 'ASL_Dataset'
    print(f'✓ Found ASL_Dataset subdirectory')

if not dataset_root.exists():
    raise FileNotFoundError(
        f'Dataset path does not exist: {dataset_root}\n'
        f'Please check the dataset structure.'
    )

# Validate class folders
class_dirs = sorted([p for p in dataset_root.iterdir() if p.is_dir()])
if not class_dirs:
    raise ValueError(f'No class directories found in {dataset_root}')

class_names = [p.name for p in class_dirs]

# Expected ASL letters (A-Y excluding J and Z which require motion)
expected_letters = [chr(code) for code in range(ord('A'), ord('Z') + 1) 
                   if chr(code) not in {'J', 'Z'}]
missing = [letter for letter in expected_letters if letter not in class_names]

print(f'\n✓ Using dataset root: {dataset_root}')
print(f'✓ Found {len(class_dirs)} class folders: {class_names}')

if missing:
    print(f'⚠ WARNING: Missing expected classes: {missing}')
    print('  Training will proceed with available classes only')
else:
    print('✓ All expected ASL classes (A-Y excluding J/Z) are present')

# Count sample images per class
total_images = 0
for class_dir in class_dirs:
    image_count = len(list(class_dir.glob('**/*.*')))
    total_images += image_count
print(f'\n✓ Total images across all classes: {total_images:,}')


In [ ]:
# Step 4: Extract wrist-centered MediaPipe landmarks from images
import cv2
import mediapipe as mp
from tqdm.auto import tqdm
import gc

print('Initializing MediaPipe Hands...')

# - Run MediaPipe Hands on every class folder and save wrist-centered landmark tensors
hands = mp.solutions.hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

labels = [d.name for d in class_dirs]
label_to_idx = {label: idx for idx, label in enumerate(labels)}
all_vectors, all_labels = [], []
class_counts = {label: 0 for label in labels}
skipped = 0
total_images = 0

print(f'Processing {len(class_dirs)} classes...\n')

for label_dir in class_dirs:
    # Get all image files (common formats)
    image_files = []
    for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
        image_files.extend(list(label_dir.glob(f'**/{ext}')))
    image_files = sorted(set(image_files))  # Remove duplicates

    if not image_files:
        print(f'⚠ No images found in {label_dir.name}')
        continue

    total_images += len(image_files)

    for img_path in tqdm(image_files, desc=f'Extracting {label_dir.name}', leave=False):
        try:
            img = cv2.imread(str(img_path))
            if img is None:
                skipped += 1
                continue

            # Convert BGR to RGB for MediaPipe
            rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            result = hands.process(rgb)

            if not result.multi_hand_landmarks:
                skipped += 1
                continue

            # Extract landmarks and apply wrist-centering normalization
            lm_array = np.array(
                [[lm.x, lm.y, lm.z] for lm in result.multi_hand_landmarks[0].landmark],
                dtype=np.float32
            )
            wrist_centered = lm_array - lm_array[0]  # Subtract wrist position
            vector = wrist_centered.flatten()  # Shape: (63,)

            all_vectors.append(vector)
            all_labels.append(label_to_idx[label_dir.name])
            class_counts[label_dir.name] += 1

        except Exception as e:
            print(f'⚠ Error processing {img_path.name}: {e}')
            skipped += 1
            continue

    # Free memory periodically
    gc.collect()

hands.close()
print('\n✓ MediaPipe processing complete')

# Convert to numpy arrays
X = np.stack(all_vectors, dtype=np.float32)
y = np.array(all_labels, dtype=np.int32)

# Save raw landmarks
np.savez(
    RAW_LANDMARKS_NPZ,
    X=X,
    y=y,
    labels=np.array(labels),
    class_counts=np.array([class_counts[label] for label in labels]),
    skipped=np.array([skipped]),
    total_images=np.array([total_images])
)

skip_ratio = skipped / max(total_images, 1)
print(f'\n📊 Extraction Summary:')
print(f'   Total images processed: {total_images:,}')
print(f'   Successfully extracted: {len(X):,} samples')
print(f'   Skipped (no hand detected): {skipped:,} ({skip_ratio:.1%})')

if skip_ratio > 0.2:
    print('⚠ WARNING: Over 20% images skipped.')
    print('  Consider: (1) Lowering detection threshold, or (2) Reviewing data quality')

print(f'\n✓ Saved raw landmarks to: {RAW_LANDMARKS_NPZ}')
print(f'\nPer-class sample counts:')
for label in sorted(class_counts.keys()):
    print(f'   {label}: {class_counts[label]:,}')


In [ ]:
# Step 5: Preprocess, augment, and create stratified train/val/test splits
from sklearn.model_selection import train_test_split

print('Loading raw landmarks...')
raw = np.load(RAW_LANDMARKS_NPZ, allow_pickle=True)
X = raw['X']
y = raw['y']
labels = raw['labels']

print(f'✓ Loaded {len(X):,} samples with {len(labels)} classes')

# Stratified deterministic splits: 75% train, 15% val, 10% test
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.25, random_state=SEED, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.4, random_state=SEED, stratify=y_temp
)

print(f'✓ Initial splits: Train={len(X_train)}, Val={len(X_val)}, Test={len(X_test)}')

# Data augmentation (optional but recommended)
ENABLE_AUGMENTATION = True  # Toggle to quickly disable augmentations

if ENABLE_AUGMENTATION:
    print('\nApplying data augmentation...')
    rng = np.random.default_rng(SEED)
    augmented_X, augmented_y = [], []
    
    for vec, label in tqdm(zip(X_train, y_train), total=len(X_train), desc='Augmenting'):
        # Augmentation 1: Add Gaussian noise (σ=0.01)
        noise = vec + rng.normal(0, 0.01, size=vec.shape)
        augmented_X.append(noise.astype(np.float32))
        augmented_y.append(label)
        
        # Augmentation 2: Horizontal flip (mirror x-coordinates)
        mirrored = vec.copy()
        mirrored[0::3] *= -1  # Flip every 3rd element (x coordinates)
        augmented_X.append(mirrored.astype(np.float32))
        augmented_y.append(label)
    
    # Combine original + augmented data
    X_train = np.concatenate([X_train, np.stack(augmented_X)], axis=0)
    y_train = np.concatenate([y_train, np.array(augmented_y)])
    
    print(f'✓ Augmentation complete: Train set expanded to {len(X_train):,} samples')
else:
    print('⚠ Augmentation disabled')

# Save processed splits
np.savez(
    PROC_NPZ,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val,
    X_test=X_test, y_test=y_test,
    labels=labels
)

print(f'\n✓ Processed splits saved to: {PROC_NPZ}')
print(f'\n📊 Final split sizes:')
print(f'   Train: {X_train.shape}')
print(f'   Val:   {X_val.shape}')
print(f'   Test:  {X_test.shape}')


In [ ]:
# Step 6: Build and train MLP classifier with callbacks and visualization
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, callbacks, optimizers

print('Loading processed data...')
proc = np.load(PROC_NPZ, allow_pickle=True)
X_train, y_train = proc['X_train'], proc['y_train']
X_val, y_val = proc['X_val'], proc['y_val']
num_classes = len(proc['labels'])

print(f'✓ Train: {X_train.shape}, Val: {X_val.shape}')
print(f'✓ Number of classes: {num_classes}')

# Build MLP architecture
print('\nBuilding model...')
model = models.Sequential([
    layers.Input(shape=(63,), name='landmark_input'),
    layers.Dense(256, activation='relu', name='dense1'),
    layers.BatchNormalization(name='bn1'),
    layers.Dropout(0.3, name='dropout1'),
    layers.Dense(128, activation='relu', name='dense2'),
    layers.Dropout(0.2, name='dropout2'),
    layers.Dense(64, activation='relu', name='dense3'),
    layers.Dense(num_classes, activation='softmax', name='output')
], name='ASL_Classifier')

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print('✓ Model compiled')
model.summary()

# Configure callbacks
checkpoint_cb = callbacks.ModelCheckpoint(
    BEST_KERAS,  # Save in modern Keras format
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

earlystop_cb = callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=7,
    restore_best_weights=True,
    verbose=1
)

reduce_lr_cb = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

print('\nStarting training...')
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=2
)

print('\n✓ Training complete!')

# Visualize training history
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
plt.title('Model Accuracy', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss', linewidth=2)
plt.plot(history.history['val_loss'], label='Val Loss', linewidth=2)
plt.title('Model Loss', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f'\n✓ Best model saved to: {BEST_KERAS}')


In [ ]:
# Step 7: Evaluate best model on test set and log detailed metrics
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

print('Loading test data and best model...')
proc = np.load(PROC_NPZ, allow_pickle=True)
X_test, y_test, labels = proc['X_test'], proc['y_test'], proc['labels']

# Load best model from training
best_model = tf.keras.models.load_model(BEST_KERAS)
print('✓ Model loaded')

# Evaluate on test set
test_loss, test_acc = best_model.evaluate(X_test, y_test, verbose=0)
print(f'\n📊 Test Set Performance:')
print(f'   Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)')
print(f'   Loss: {test_loss:.4f}')

# Generate predictions
y_probs = best_model.predict(X_test, verbose=0)
y_pred = y_probs.argmax(axis=1)

# Classification report
print('\n' + '='*70)
print('CLASSIFICATION REPORT')
print('='*70)
print(classification_report(y_test, y_pred, target_names=labels, digits=4))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=labels, columns=labels)

# Visualize confusion matrix
plt.figure(figsize=(12, 10))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues', cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - Test Set', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

# Save sample predictions
sample_df = pd.DataFrame({
    'true_label': [labels[idx] for idx in y_test[:50]],
    'pred_label': [labels[idx] for idx in y_pred[:50]],
    'confidence': y_probs[:50].max(axis=1),
    'correct': [y_test[i] == y_pred[i] for i in range(min(50, len(y_test)))]
})

sample_csv_path = os.path.join(OUTPUT_DIR, 'sample_predictions.csv')
sample_df.to_csv(sample_csv_path, index=False)
print(f'\n✓ Sample predictions saved to: {sample_csv_path}')
print(f'\nFirst 10 predictions:')
print(sample_df.head(10).to_string(index=False))


In [ ]:
# Step 8: Export model in multiple formats (Keras, SavedModel, labels)
import os
import json
import shutil
import numpy as np
import tensorflow as tf

print('Preparing to export model artifacts...')

# Clean up previous exports to avoid conflicts
if os.path.exists(SAVED_MODEL_DIR):
    shutil.rmtree(SAVED_MODEL_DIR)
    print('✓ Cleaned old SavedModel directory')

if os.path.exists(TFJS_DIR):
    shutil.rmtree(TFJS_DIR)
    print('✓ Cleaned old TFJS directory')

os.makedirs(TFJS_DIR, exist_ok=True)

# Load the best model
print(f'\nLoading best model from: {BEST_KERAS}')
model = tf.keras.models.load_model(BEST_KERAS)
print('✓ Model loaded successfully')

# Export 1: SavedModel format (standard TensorFlow format)
print(f'\nExporting to SavedModel format...')
model.export(SAVED_MODEL_DIR)  # TF 2.16+ uses export() instead of save()
print(f'✓ SavedModel saved: {SAVED_MODEL_DIR}')

# Verify SavedModel structure
if os.path.exists(os.path.join(SAVED_MODEL_DIR, 'saved_model.pb')):
    print('  ✓ saved_model.pb exists')
if os.path.exists(os.path.join(SAVED_MODEL_DIR, 'variables')):
    print('  ✓ variables/ directory exists')

# Export 2: Create labels mapping JSON
print(f'\nCreating labels mapping...')
proc = np.load(PROC_NPZ, allow_pickle=True)
labels = proc['labels']
label_map = {int(idx): str(label) for idx, label in enumerate(labels)}

with open(LABELS_JSON, 'w', encoding='utf-8') as f:
    json.dump(label_map, f, indent=2, ensure_ascii=False)
print(f'✓ Labels saved: {LABELS_JSON}')
print(f'  Classes: {list(label_map.values())}')

# Summary of exported artifacts
print('\n' + '='*70)
print('EXPORT SUMMARY')
print('='*70)

def get_size_mb(path):
    """Calculate directory or file size in MB"""
    if os.path.isfile(path):
        return os.path.getsize(path) / (1024 * 1024)
    total = 0
    for root, dirs, files in os.walk(path):
        for f in files:
            total += os.path.getsize(os.path.join(root, f))
    return total / (1024 * 1024)

print(f'\n✓ Keras Model (.keras):     {get_size_mb(BEST_KERAS):.2f} MB')
print(f'✓ SavedModel (directory):   {get_size_mb(SAVED_MODEL_DIR):.2f} MB')
print(f'✓ Labels JSON:              {get_size_mb(LABELS_JSON):.3f} MB')

print('\n🎉 All model artifacts exported successfully!')
print(f'\nNext step: Run the TFJS conversion cell to create browser-compatible model')


In [ ]:
# Step 9: Convert SavedModel to TensorFlow.js format for browser deployment
import subprocess
import sys
import os

print('Converting model to TensorFlow.js format...')
print(f'Source: {SAVED_MODEL_DIR}')
print(f'Target: {TFJS_DIR}\n')

try:
    # Check if tensorflowjs_converter is available
    check_result = subprocess.run(
        ['which', 'tensorflowjs_converter'],
        capture_output=True,
        text=True
    )
    
    if check_result.returncode != 0:
        print('⚠ tensorflowjs_converter not found in PATH, trying pip installation...')
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'tensorflowjs'], check=True)
        print('✓ tensorflowjs installed')
    
    # Run conversion
    print('Running conversion (this may take 1-2 minutes)...')
    result = subprocess.run([
        'tensorflowjs_converter',
        '--input_format=tf_saved_model',
        '--output_format=tfjs_graph_model',
        '--signature_name=serving_default',
        '--strip_debug_ops=True',
        SAVED_MODEL_DIR,
        TFJS_DIR
    ], check=True, capture_output=True, text=True)
    
    if result.stdout:
        print(result.stdout)
    
    print('\n✓ TFJS conversion complete!')
    
    # List generated files
    print(f'\n📦 Generated TFJS artifacts:')
    tfjs_files = sorted(os.listdir(TFJS_DIR))
    total_size = 0
    
    for item in tfjs_files:
        item_path = os.path.join(TFJS_DIR, item)
        size_mb = os.path.getsize(item_path) / (1024 * 1024)
        total_size += size_mb
        print(f'   • {item:<30} {size_mb:>8.2f} MB')
    
    print(f'\n   Total TFJS model size: {total_size:.2f} MB')
    
    # Verify critical files
    model_json = os.path.join(TFJS_DIR, 'model.json')
    if os.path.exists(model_json):
        print(f'\n✓ model.json exists - ready for web deployment')
        print(f'  Load in browser with: tf.loadGraphModel("path/to/model.json")')
    else:
        print('\n⚠ WARNING: model.json not found!')
        
except FileNotFoundError as e:
    print(f'❌ ERROR: Command not found - {e}')
    print('Make sure tensorflowjs is installed:')
    print('  !pip install tensorflowjs')
    sys.exit(1)
    
except subprocess.CalledProcessError as exc:
    print(f'❌ TFJS conversion failed with exit code {exc.returncode}')
    print(f'\nSTDOUT:\n{exc.stdout}')
    print(f'\nSTDERR:\n{exc.stderr}')
    print('\nTroubleshooting:')
    print('1. Check that SavedModel format is valid')
    print('2. Try updating tensorflowjs: !pip install -U tensorflowjs')
    print('3. Verify sufficient disk space in Drive')
    raise

except Exception as e:
    print(f'❌ Unexpected error: {e}')
    raise

print('\n🎉 Model is ready for browser deployment!')


# 🌐 Runtime Integration Guide

This cell provides reference code for deploying the trained model in a web browser.

## Browser Deployment Steps

1. **Install TensorFlow.js and Hand Detection**
```bash
npm install @tensorflow/tfjs
npm install @tensorflow-models/hand-pose-detection
```

2. **Load the TFJS model and labels**
```javascript
// Load the classifier model
const model = await tf.loadGraphModel('/path/to/tfjs_model/model.json');

// Load labels
const labelsResponse = await fetch('/path/to/labels.json');
const labels = await labelsResponse.json();
```

3. **Initialize MediaPipe Hands detector**
```javascript
const detector = await handPoseDetection.createDetector(
  handPoseDetection.SupportedModels.MediaPipeHands,
  {
    runtime: 'tfjs',
    maxHands: 1,
    modelType: 'full'
  }
);
```

4. **Process video frames and classify**
```javascript
async function classifyFrame(videoElement) {
  // Detect hand landmarks
  const hands = await detector.estimateHands(videoElement, {
    flipHorizontal: true
  });
  
  if (!hands || hands.length === 0) {
    return null; // No hand detected
  }
  
  // Get 3D keypoints (21 landmarks)
  const keypoints = hands[0].keypoints3D;
  
  // Apply wrist-centering normalization
  const wrist = keypoints[0];
  const wristCentered = keypoints.flatMap(pt => [
    pt.x - wrist.x,
    pt.y - wrist.y,
    pt.z - wrist.z
  ]);
  
  // Create input tensor [1, 63]
  const inputTensor = tf.tensor2d([wristCentered], [1, 63]);
  
  // Run inference
  const predictions = model.predict(inputTensor);
  const probabilities = await predictions.data();
  
  // Get top prediction
  const maxIdx = probabilities.indexOf(Math.max(...probabilities));
  const confidence = probabilities[maxIdx];
  
  // Clean up tensors
  inputTensor.dispose();
  predictions.dispose();
  
  return {
    letter: labels[maxIdx],
    confidence: confidence
  };
}
```

5. **Implement smoothing (optional but recommended)**
```javascript
// Use majority voting over N frames to reduce jitter
class PredictionSmoother {
  constructor(windowSize = 5) {
    this.window = [];
    this.windowSize = windowSize;
  }
  
  addPrediction(letter, confidence) {
    if (confidence > 0.7) { // Confidence threshold
      this.window.push(letter);
      if (this.window.length > this.windowSize) {
        this.window.shift();
      }
    }
  }
  
  getStablePrediction() {
    if (this.window.length < 3) return null;
    
    // Count occurrences
    const counts = {};
    this.window.forEach(letter => {
      counts[letter] = (counts[letter] || 0) + 1;
    });
    
    // Return most frequent
    return Object.keys(counts).reduce((a, b) => 
      counts[a] > counts[b] ? a : b
    );
  }
}
```

## Important Notes

- **Wrist-centering is critical**: Subtract landmark[0] from all landmarks before inference
- **Input shape**: Model expects `[1, 63]` (21 landmarks × 3 coordinates, flattened)
- **Coordinate order**: Each landmark is `[x, y, z]`
- **CORS**: Host TFJS files on same domain as your web app to avoid CORS issues
- **Performance**: Use `requestAnimationFrame` for smooth real-time detection

## Hosting

Copy the `tfjs_model/` directory and `labels.json` to your web server's static assets folder.


In [ ]:
# Step 10: Generate README and list all exported artifacts
import os
from datetime import datetime

print('Generating artifact manifest...\n')

# Create comprehensive README
readme_path = os.path.join(OUTPUT_DIR, 'README.txt')
readme_content = f"""
================================================================================
ASL SIGN LANGUAGE LANDMARK CLASSIFIER - TRAINING ARTIFACTS
================================================================================

Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Training Framework: TensorFlow {tf.__version__}
Python Version: {sys.version.split()[0]}

================================================================================
DATASET INFORMATION
================================================================================

Dataset: {DATASET_KAGGLE}
Source: Kaggle (via kagglehub)
Expected Classes: 24 ASL letters (A-Y, excluding J and Z which require motion)

================================================================================
MODEL ARCHITECTURE
================================================================================

Type: Sequential MLP (Multi-Layer Perceptron)
Input: 63 features (21 MediaPipe hand landmarks × 3 coordinates, wrist-centered)

Layers:
  1. Dense(256, relu) + BatchNormalization + Dropout(0.3)
  2. Dense(128, relu) + Dropout(0.2)
  3. Dense(64, relu)
  4. Dense(num_classes, softmax)

Optimizer: Adam (lr=1e-3)
Loss: Sparse Categorical Crossentropy

================================================================================
TRAINING CONFIGURATION
================================================================================

Random Seed: {SEED}
Train/Val/Test Split: 75% / 15% / 10%
Data Augmentation: Gaussian noise + Horizontal flip
Batch Size: 32
Max Epochs: 100
Early Stopping: Patience=7 (monitor val_accuracy)

================================================================================
EXPORTED ARTIFACTS
================================================================================

1. best_model.keras
   - Modern Keras format (TF 2.16+)
   - Best model based on validation accuracy
   - Use: tf.keras.models.load_model('best_model.keras')

2. saved_model/ (directory)
   - Standard TensorFlow SavedModel format
   - Contains: saved_model.pb + variables/
   - Use: tf.saved_model.load('saved_model')

3. tfjs_model/ (directory)
   - TensorFlow.js graph model format
   - For browser-based inference
   - Load: tf.loadGraphModel('tfjs_model/model.json')

4. labels.json
   - Class index to letter mapping
   - Format: {{"0": "A", "1": "B", ...}}

5. hand_landmarks_raw.npz
   - Raw extracted landmarks from all images
   - Arrays: X, y, labels, class_counts, skipped, total_images

6. hand_landmarks_processed.npz
   - Preprocessed and augmented training data
   - Arrays: X_train, y_train, X_val, y_val, X_test, y_test, labels

7. sample_predictions.csv
   - First 50 test set predictions
   - Columns: true_label, pred_label, confidence, correct

================================================================================
USAGE NOTES
================================================================================

Preprocessing Pipeline:
  1. Extract 21 hand landmarks using MediaPipe Hands (static_image_mode=True)
  2. Apply wrist-centering: subtract landmark[0] from all landmarks
  3. Flatten to [63] vector
  4. Feed to classifier

For Real-Time Inference:
  - Use same preprocessing: detect hands → wrist-center → flatten → predict
  - Apply confidence threshold (recommended: 0.7)
  - Use temporal smoothing (e.g., 5-frame majority voting) to reduce jitter

Browser Deployment:
  - Use @tensorflow-models/hand-pose-detection for landmark extraction
  - Load tfjs_model/model.json with tf.loadGraphModel()
  - Ensure wrist-centering preprocessing matches training

================================================================================
FILE PATHS
================================================================================

Output Directory: {OUTPUT_DIR}

Keras Model:      {BEST_KERAS}
SavedModel:       {SAVED_MODEL_DIR}
TFJS Model:       {TFJS_DIR}
Labels:           {LABELS_JSON}
Raw Landmarks:    {RAW_LANDMARKS_NPZ}
Processed Data:   {PROC_NPZ}

================================================================================
"""

with open(readme_path, 'w', encoding='utf-8') as f:
    f.write(readme_content)

print(f'✓ README saved to: {readme_path}\n')

# List all artifacts with sizes
print('='*70)
print('ARTIFACT INVENTORY')
print('='*70)

def list_output_files(base_dir):
    """Recursively list all files with sizes"""
    items = []
    for root, dirs, files in os.walk(base_dir):
        for file in sorted(files):
            path = os.path.join(root, file)
            size_mb = os.path.getsize(path) / (1024 * 1024)
            rel_path = os.path.relpath(path, base_dir)
            items.append((rel_path, size_mb))
    return sorted(items)

total_size = 0
for rel_path, size_mb in list_output_files(OUTPUT_DIR):
    total_size += size_mb
    print(f'{rel_path:<50} {size_mb:>10.2f} MB')

print('='*70)
print(f'{'TOTAL SIZE':<50} {total_size:>10.2f} MB')
print('='*70)

print(f'\n✅ All artifacts successfully saved to Google Drive!')
print(f'\n📂 Access your files at: {OUTPUT_DIR}')
print(f'\n💡 Tip: Download the entire folder to use in your web application')
